A sample of a code of the VGG16 signature verification model trained on CEDAR dataset and the sythetic attack process

In [ ]:
import cv2                 # working with, mainly resizing, images
import numpy as np         # dealing with arrays
import os                  # dealing with directories
from random import shuffle
import numpy as np
from keras.utils import to_categorical
from keras.models import Sequential, Model, load_model
from keras import applications
from keras import optimizers
from keras.layers import Dropout, Flatten, Dense
import tensorflow as tf
import matplotlib.pyplot as plt
import sklearn
from sklearn import metrics 

In [ ]:
path=os.getcwd()
#link to the datasets. The images must be converted into numpy arrays with size 150x220 and saved in the files. 
#cedar dataset has 55 contributors, each has 24 genuine samples and 24 forgery samples
train_real_path=path+"/real_CEDAR.npy"
train_fake_path=path+"/forg_CEDAR.npy"
syn_real_path= path+"/synthetic_real_CEDAR.npy"
syn_fake_path= path+"/synthetic_forg_CEDAR.npy"

In [ ]:
def init_model():
  #the model runs on GPU
  with tf.device('/device:GPU:0'):
    base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=(150, 220, 3))
    for layer in base_model.layers:
      layer.trainable = False

    add_model = Sequential()
    add_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    add_model.add(Dense(256, activation='relu'))
    add_model.add(Dense(2, activation='sigmoid'))
    model = Model(inputs=base_model.input, outputs=add_model(base_model.output))
    model.compile(loss='binary_crossentropy', optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
    return model

In [ ]:
with open(train_fake_path, 'rb') as f:
    fake=np.load(f)
with open(train_real_path, 'rb') as f:
    real=np.load(f)
with open(syn_real_path, 'rb') as f:
      syn_real=np.load(f)
with open(syn_fake_path, 'rb') as f:
      syn_fake=np.load(f) 

In [ ]:
def data_generate_cedar(index,syn=False):
  #generate train and test features for an individual 
  train_features = []
  train_labels = []
  test_features = []
  test_labels = []
  syn_real_features = []
  syn_real_labels = []
  syn_fake_features = []
  syn_fake_labels = []

  #the real/forg human dataset would be arranged sequentially - first sample of the first user, second sample of the first user, and vice versa
  for j in range (0,16) :
    train_features.append(real[index*24+j])
    train_labels.append(1)
    train_features.append(fake[index*24+j])
    train_labels.append(0)
  for j in range (16,24) :
    test_features.append(real[index*24+j])
    test_labels.append(1)
    test_features.append(fake[index*24+j])
    test_labels.append(0)
  if syn: 
  #for each human sample we generate 9 synthetic samples, 
  #the arrangement is similar to human datasets - 9 synthetic from first sample of the first user, 9 from the second sample of the first user, and vice versa
  for j in range (0,16) :
    for i in range(16,24):
      for j in range(9):
        syn_real_features.append(syn_real[index*24*9+i*9+j])
        syn_real_labels.append(0)
        syn_fake_features.append(syn_fake[index*24*9+i*9+j])
        syn_fake_labels.append(0)
  train_features = np.asarray(train_features).astype('float32')
  test_features = np.asarray(test_features).astype('float32')
  syn_real_features = np.asarray(syn_real_features).astype('float32')
  syn_fake_features = np.asarray(syn_fake_features).astype('float32')
  train_features = train_features/ 255.
  test_features = test_features / 255.
  syn_real_features = syn_real_features /255.
  syn_fake_features = syn_fake_features /255.
  train_labels = np.array(train_labels)
  test_labels = np.array(test_labels)
  syn_real_labels = np.array(syn_real_labels)
  syn_fake_labels = np.array(syn_fake_labels)
  train_labels = to_categorical(train_labels)
  return train_features,train_labels,test_features,test_labels,syn_real_features,syn_real_labels,syn_fake_features,syn_fake_labels

In [ ]:
def make_prediction(data,model):
  tp = model.predict(data)
  #output has 2 values - probability of being 0 (fake) and probability of being 1 (real)
  output=[]
  for i in tp:
    output.append(i[1])
  return output

In [ ]:
def acc(data,labels):
  output =[]
  for i in data:
    #threshold 0.5
    if i>0.5: #real 
      output.append(1)
    else:
      output.append(0)
  return sklearn.metrics.accuracy_score(output,labels)

In [ ]:
def far_frr_cal(data,compare):
  #calculation of far and frr through out the threshold
  real = 0
  fake = 0
  frr = []
  far =[]
  for i in range (11):
    frr.append(0)
    far.append(0)
  for i in range (len(data)):
    if compare[i]==1:
      real+=1
    else:
      fake+=1
  for i in range (len(data)):
    for j in range(11):
      tp = 1
      if data[i] <= j*0.1:
        tp = 0
      if tp!=compare[i]:
        if compare[i]==0:
          far[j]+=1
        else:
          frr[j]+=1
  for i in range (11):
    frr[i]/=real/100
    far[i]/=fake/100
  return [far,frr]
         
def far_cal(data,compare):
  far =[]
  for i in range (11):
    far.append(0)
  for i in range (len(data)):
    for j in range(11):
      tp = 1
      if data[i] <= j*0.1:
        tp = 0
      if tp==1:
        far[j]+=1
  for i in range (11):
    far[i]/=len(data)/100
  return far
         


In [ ]:
def evaluate_person(index):
  train_features,train_labels,test_features,test_labels,syn_real_features,syn_real_labels,syn_fake_features,syn_fake_labels = data_generate_cedar(index,True)
  batch_size = 32
  epochs = 30
  model = init_model()
  history = model.fit(train_features, train_labels, batch_size=batch_size,epochs=epochs,verbose=1)  
  human_test = make_prediction(test_features,model)
  syn_real_test = make_prediction(syn_real_features,model)
  syn_fake_test = make_prediction(syn_fake_features,model)
  return acc(human_test,test_labels), acc(syn_real_test ,syn_real_labels),acc(syn_fake_test,syn_fake_labels), far_frr_cal(human_test,test_labels),far_cal(syn_real_test ,syn_real_labels),far_cal(syn_fake_test ,syn_fake_labels)

In [ ]:
import pandas as pd
import csv

0

In [ ]:
human_frr = []
for i in range (11):
  human_frr.append(0)
human_far = []
for i in range (11):
  human_far.append(0)
syn_real_far = []
for i in range (11):
  syn_real_far.append(0)
syn_fake_far = []
for i in range (11):
  syn_fake_far.append(0)
for i in range (0, no_contrib):
  print('----------- evaluating person '+str(i+1)+'--------------')
  #
  a,b,c,x,y,z = evaluate_person(i)
  #"human_far","human_frr","synt_real_far", "synt_fake_far"
  print([x[0][6],x[1][6],y[6],z[6]])
